# Projet maison n° 3

In [8]:
# imports
import pandas as pd

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [9]:
from numpy import int64
# names us
#import re
import glob
import os

def df_names_us():
    df = pd.DataFrame()
    df1 = pd.DataFrame()
    f2 = []
    for filename in glob.glob('names/*.txt'):
        y=os.path.basename('names/'+filename)
        annee=y[3:7]
    
        with open(os.path.join(os.getcwd(), filename), 'r') as f:
            dfx = pd.read_csv(f, sep=",", header=None, 
                 names=["name", "gender","births"],dtype=object).assign(year=annee)  
        #n=len(dfx)
        #for i in range(n):
            #f2.append(annee)
            df1=pd.concat([df1,dfx], axis=0)

    df=df1[["year","name","gender","births"]].sort_values(by=["year"],ascending=True)
    df.index=pd.RangeIndex(start=0,stop=len(df))
    return df

df=df_names_us()
df

,year,name,gender,births
0,1880,Asberry,M,6
1,1880,Vern,M,19
2,1880,Stewart,M,19
3,1880,Randolph,M,19
4,1880,Lucien,M,19
...,...,...,...,...
2052776,2021,Dansby,M,32
2052777,2021,Darvin,M,32
2052778,2021,Dayson,M,32
2052779,2021,Dio,M,32


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [10]:
# names fr
import numpy as np
import pandas as pd
import re
def df_names_fr():
    df = pd.DataFrame()
    df=pd.read_csv('nat2021.csv',sep=";" ,
                 names=["gender", "name","year","births"],dtype=object, keep_default_na=False).loc[1:]
    df=df[["year","name","gender","births"]]
    mapping = {'1': 'M', '2': 'F', 'F': 'F', 'M': 'M'}
    df['gender'] = df['gender'].map(mapping)
    df=df[df["name"].apply(lambda x: str(x)).map(len)>=2]
    df["name"]=df["name"].str.title()
    df.drop(df.loc[df['name']=='_Prenoms_Rares'].index, inplace=True)
    df.drop(df.loc[df['year']=='XXXX'].index, inplace=True)
    df.dropna()
    df.index=pd.RangeIndex(start=0,stop=len(df))

    return df
df1=df_names_fr()
df1

,year,name,gender,births
0,2009,Aadam,M,4
1,2014,Aadam,M,3
2,2015,Aadam,M,3
3,2016,Aadam,M,4
4,2017,Aadam,M,4
...,...,...,...,...
648325,2017,Zyneb,F,6
648326,2018,Zyneb,F,5
648327,2019,Zyneb,F,7
648328,2020,Zyneb,F,8


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [12]:
# taux de change
import re
def df_taux_change(devises):
    df = pd.DataFrame([], columns=devises)
    
    # Chargement des donnees
    df=pd.read_csv('Webstat_Export_20220923.csv',sep=";", skiprows = range(1,6),low_memory=False)
    df = df.rename(columns={"Titre :": "Date"})
    df1=df.iloc[:,1:]
    #df.set_index("Date ")
    for name , data in df1.iteritems():
        if any(s in name for s in devises):
            df = df.rename(columns={name: name[-4:-1]})
        else:
            if (name != "Date"):
                df = df.drop(columns=[name])
                
    # On retiend uniquement les colonnes se terminant par un code ISO3 entre parenthèses
    pattern='\((.*?)\)'
    colonns = re.findall(pattern,str(df.iloc[0]))
    
    # Nettoyage
    df.index = pd.to_datetime(df["Date"], format = "%d/%m/%Y")
    df=df.iloc[:,1:]
    df = df.sort_values(by = ["Date"], ascending=[True])
    cols=dict(zip(list(range(len(colonns))), colonns))
    
    df = df.apply(lambda x: x.str.replace(',','.',regex=True))
    df = df.apply(lambda x: x.replace({'-': None}))
    df.dropna(axis = 0,inplace = True)
    
    df = (df.iloc[:,0:3]).astype(float)
    

    return df

df_taux_change(["CHF", "GBP", "USD"])

,CHF,GBP,USD
Date,,,
1999-02-01,1.6089,0.69040,1.1338
1999-03-01,1.5930,0.68290,1.0986
1999-04-01,1.5957,0.66900,1.0772
1999-06-01,1.5928,0.65050,1.0434
1999-07-01,1.6043,0.65020,1.0264
...,...,...,...
2022-03-01,1.0247,0.83290,1.1162
2022-04-01,1.0217,0.84145,1.1052
2022-06-01,1.0305,0.85158,1.0712


### Tests

In [13]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [14]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [15]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 17.780s

OK
